In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from itertools import chain
from langchain_core.prompts import PromptTemplate


email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model='gpt-4o')

chain = prompt | llm

answer = chain.stream({'email_conversation': email_conversation})

output = stream_response(answer, return_output=True)

print(output)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무가 바이크코퍼레이션을 대표하여 이은채 대리에게 연락.
2. "ZENESIS" 자전거에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함).
3. 유통 전략과 마케팅 계획을 구체화하기 위한 정보 필요.
4. 협력 가능성 논의를 위한 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서.이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무가 바이크코퍼레이션을 대표하여 이은채 대리에게 연락.
2. "ZENESIS" 자전거에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함).
3. 유통 전략과 마케팅 계획을 구체화하기 위한 정보 필요.
4. 협력 가능성 논의를 위한 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서.


In [3]:
# using pydantic output parser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions())

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm

response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

output = stream_response(response, return_output=True)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```
```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikec

In [4]:
# we can parse the results using PydanticOutputParser

structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 언급했습니다. 또한, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.' date='1월 15일 화요일 오전 10시'


In [5]:
# Adding the parser to the chain might prevent the use of the stream function.

chain = prompt | llm | parser

response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용 중 주요 내용을 추출해 주세요.",
    }
)

response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 언급했습니다. 또한, 유통 전략과 마케팅 계획을 구체화하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 화요일 오전 10시')

In [6]:
# using comma separated list output parser

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    partial_variables={"format_instructions": format_instructions},
)
  
model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

chain.invoke({'subject': "대한민국 관광명소"})

for s in chain.stream({'subject':"대한민국 관광명소"}):
    print(s)

['경복궁']
['인사동']
['부산 해운대해수욕장']
['제주도']
['남산타워']
